In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.stats import spearmanr, kendalltau

In [2]:
df = pd.read_csv("marketing_advanced.csv", encoding="euc-kr")

In [3]:
df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,age,gender,country,...,category2,price,delivery_delay_days,approval_delay_minutes,weight_kg,volume_cm3,review_score,marketing_info_agree,agegroup,product_id
0,e22acc9c116caa3f2b7121bbb380d08e,0000366f3b9a7992bf8c76cfdf3221e2,delivered,2024-10-10 16:29,2024-10-10 19:21,2024-10-16 20:48,2024-10-21,56,남성,핀란드,...,건강식품,62.17,-5.0,15.0,5,11845.97,5,y,50대,d2e1b02d25294e6f
1,3594e05a005ac4d06a72673270ef9ec9,0000b849f77a49e4a4ce2b2a4ca5be3f,delivered,2024-10-07 17:47,2024-10-07 19:14,2024-10-10 18:02,2024-10-15,61,여성,한국,...,건강식품,57.90,-5.0,434.0,1,25981.89,4,y,50대,d2e1b02d25294e6f
2,b33ec3b699337181488304f362a6b734,0000f46a3911fa3c0805444483337064,delivered,2023-08-10 16:35,2023-08-10 19:29,2023-09-05 14:38,2023-09-07,51,남성,필리핀,...,건강식품,60.34,-2.0,0.0,0,26478.17,2,n,50대,d2e1b02d25294e6f
3,41272756ecddd9a9ed0180413cc22fb6,0000f6ccb0745a6a4b88665a16c9f078,delivered,2024-03-12 15:03,2024-03-12 17:00,2024-04-01 21:23,2024-04-13,34,여성,덴마크,...,음향가전,128.17,-12.0,20.0,10,208064.64,4,y,30대,f9614bb192ee4050
4,d957021f1127559cd947b62533f484f7,0004aac84e0df4da2b147fca70cf8255,delivered,2024-04-14 13:49,2024-04-14 15:41,2024-04-27 23:08,2024-05-05,42,남성,벨기에,...,주방가전,110.15,-8.0,21.0,5,185186.95,2,y,40대,76a306b4cd254f91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104894,725cf8e9c24e679a8a5a32cb92c9ce1e,fffcf5a5ff07b0908bd4e2dbc735a684,delivered,2023-11-08 13:35,2023-11-08 15:45,2023-12-06 9:22,2024-01-02,40,여성,브라질,...,비데,100.56,-27.0,15.0,1,228394.78,2,y,40대,4d1488c50d3f4b25
104895,c71b9252fd7b3b263aaa4cb09319a323,fffea47cd6d3cc0a88bd621562a9d061,delivered,2024-05-10 12:45,2024-05-10 15:17,2024-06-09 22:28,2024-06-12,39,여성,이탈리아,...,건강식품,58.66,-3.0,9.0,5,17850.64,2,y,30대,d2e1b02d25294e6f
104896,fdc45e6c7555e6cb3cc0daca2557dbe1,ffff371b4d645b6ecea244b27531430a,delivered,2023-07-07 4:15,2023-07-07 5:23,2023-07-22 12:45,2023-08-24,46,남성,폴란드,...,비데,96.40,-30.0,13.0,10,234820.57,2,y,40대,4d1488c50d3f4b25
104897,94d3ee0bc2a0af9d4fa47a4d63616e8d,ffff5962728ec6157033ef9805bacc48,delivered,2024-10-02 20:34,2024-10-02 22:38,2024-10-14 11:54,2024-10-28,48,여성,독일,...,음향가전,129.60,-14.0,41.0,25,189670.79,2,y,40대,f9614bb192ee4050


In [4]:
df['category1'].unique()

array(['식품', '전자제품', '스포츠용품', '의류', '가구', '화장품'], dtype=object)

In [5]:
order_per_user = df.groupby(['category1', 'customer_id'])['order_id'].count().reset_index()
result = order_per_user.groupby('category1')['order_id'].mean()
result

category1
가구       1.035328
스포츠용품    1.045665
식품       1.054427
의류       1.045920
전자제품     1.042078
화장품      1.045377
Name: order_id, dtype: float64

In [6]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df_sorted = df.sort_values(['customer_id', 'category1', 'order_purchase_timestamp'])
df_sorted['reorder_gap'] = df_sorted.groupby(['customer_id', 'category1'])['order_purchase_timestamp'].diff().dt.days
result = df_sorted.groupby('category1')['reorder_gap'].mean()
print(result)

category1
가구       12.923445
스포츠용품    19.791418
식품       18.058696
의류       16.607801
전자제품     17.272490
화장품      18.613309
Name: reorder_gap, dtype: float64


### 카테고리1 별 평균 재주문까지 걸린 일수

In [7]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df_sorted = df.sort_values(['customer_id', 'category1', 'order_purchase_timestamp'])
df_sorted['reorder_gap'] = df_sorted.groupby(['customer_id', 'category1'])['order_purchase_timestamp'].diff().dt.days
mean_gap = df_sorted.groupby('category1')['reorder_gap'].mean()
median_gap = df_sorted.groupby('category1')['reorder_gap'].median()
print("카테고리1별 재주문까지 걸린 평균 일수:\n", mean_gap)
print("\n카테고리1별 재주문까지 걸린 중앙값(일수):\n", median_gap)

카테고리1별 재주문까지 걸린 평균 일수:
 category1
가구       12.923445
스포츠용품    19.791418
식품       18.058696
의류       16.607801
전자제품     17.272490
화장품      18.613309
Name: reorder_gap, dtype: float64

카테고리1별 재주문까지 걸린 중앙값(일수):
 category1
가구       0.0
스포츠용품    0.0
식품       0.0
의류       0.0
전자제품     0.0
화장품      0.0
Name: reorder_gap, dtype: float64


In [8]:
df['category2'].unique()

array(['건강식품', '음향가전', '주방가전', '낚세', '캠핑', '신선식품', '파티의류', '가공식품',
       '책상/책장', '침대', '바디케어', '거실장/테이블', '건강가전', '유아의류', '비데', '스킨케어',
       '스포츠의류', '여성의류', '등산', '네일케어', '골프', '남성의류', '메이크업', '마스크/팩',
       '선케어', '세탁기/건조기', '청소기', '옷장/행거', '남성화장품', '헤어케어', '수납장/선반', '화장대',
       '냉장고', '1인가구', '향수', '쇼파', '야외가구'], dtype=object)

In [9]:
order_per_user2 = df.groupby(['category2', 'customer_id'])['order_id'].count().reset_index()
result2 = order_per_user2.groupby('category2')['order_id'].mean()
result2

category2
1인가구       1.003135
가공식품       1.018100
거실장/테이블    1.000000
건강가전       1.005549
건강식품       1.020892
골프         1.010947
낚세         1.014850
남성의류       1.008763
남성화장품      1.006512
냉장고        1.006384
네일케어       1.001830
등산         1.013020
마스크/팩      1.003610
메이크업       1.028643
바디케어       1.002801
비데         1.005714
선케어        1.004437
세탁기/건조기    1.004729
쇼파         1.001404
수납장/선반     1.000000
스킨케어       1.026744
스포츠의류      1.008224
신선식품       1.017214
야외가구       1.002857
여성의류       1.011305
옷장/행거      1.002903
유아의류       1.009711
음향가전       1.005253
주방가전       1.005247
책상/책장      1.001531
청소기        1.004915
침대         1.002890
캠핑         1.013866
파티의류       1.008008
향수         1.006272
헤어케어       1.005381
화장대        1.003140
Name: order_id, dtype: float64

### 카테고리2 별로 재주문까지 걸린 일수

In [10]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df_sorted = df.sort_values(['customer_id', 'category2', 'order_purchase_timestamp'])
df_sorted['reorder_gap'] = df_sorted.groupby(['customer_id', 'category2'])['order_purchase_timestamp'].diff().dt.days
mean_gap_2 = df_sorted.groupby('category2')['reorder_gap'].mean()
median_gap_2 = df_sorted.groupby('category2')['reorder_gap'].median()
print(mean_gap_2)
print(median_gap_2)

category2
1인가구        0.000000
가공식품       14.031847
거실장/테이블          NaN
건강가전       35.866667
건강식품       13.162162
골프          9.960784
낚세         13.633803
남성의류        8.527778
남성화장품       5.428571
냉장고         1.470588
네일케어        0.000000
등산         36.096774
마스크/팩       5.500000
메이크업       17.108696
바디케어        0.000000
비데          2.375000
선케어        28.200000
세탁기/건조기    42.692308
쇼파          0.000000
수납장/선반           NaN
스킨케어        9.021739
스포츠의류       8.000000
신선식품       21.160000
야외가구        0.000000
여성의류       17.854167
옷장/행거       0.000000
유아의류       32.341463
음향가전        7.714286
주방가전        8.928571
책상/책장       0.000000
청소기        13.384615
침대         14.000000
캠핑         16.954545
파티의류       11.060606
향수         30.285714
헤어케어       75.500000
화장대        24.000000
Name: reorder_gap, dtype: float64
category2
1인가구        0.0
가공식품        0.0
거실장/테이블     NaN
건강가전        0.0
건강식품        0.0
골프          0.0
낚세          0.0
남성의류        0.0
남성화장품       0.0
냉장고         0.0
네일케어     

### 가장 오래전에 주문한 사람

In [11]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

In [12]:
oldest_order = df.loc[df['order_purchase_timestamp'].idxmin()]

In [13]:
df_sorted = df.sort_values(by='order_purchase_timestamp', ascending=True)

In [14]:
df_sorted[['customer_id', 'order_purchase_timestamp']]

,customer_id,order_purchase_timestamp
75400,b7d76e111c89f7ebf14761390f0f7d17,2023-02-04 09:05:00
75399,b7d76e111c89f7ebf14761390f0f7d17,2023-02-04 18:35:00
29616,4854e9b3feff728c13ee5fc7d1547e92,2023-02-05 19:39:00
5996,0eb1ee9dba87f5b36b4613a65074337c,2023-03-02 15:58:00
6952,10e89fd8e5c745f81bec101207ba4d7d,2023-03-03 06:15:00
...,...,...
13892,21dbe8eabd00b34492a939c540e2b1a7,2025-01-29 15:35:00
15024,24ac2b4327e25baf39f2119e4228976a,2025-01-29 18:22:00
74222,b4dcade04bc548b7e3b0243c801f8c26,2025-01-29 19:31:00
1693,0421e7a23f21e5d54efed456aedbc513,2025-01-29 21:38:00


### 주문량이 많은 순서

In [15]:
order_count = df['customer_id'].value_counts().reset_index()
order_count.columns = ['customer_id', 'order_count']

In [16]:
order_count

,customer_id,order_count
0,8d50f5eadf50201ccdcedfb9e2ac8455,16
1,3e43e6105506432c953e165fb2acf44c,13
2,41a3b256cc497dc952a815b848345cbc,12
3,1b6c7548a2a1f9037c1fd3ddfed95f33,10
4,ca77025e7201e3b30c44b472ff346268,9
...,...,...
95414,58830e73978eab27a86d804befbfa3a7,1
95415,5882e10ed75dbb86ed6b62bddb146e5e,1
95416,5880e398144a5fc8dbcd185521e3f6bc,1
95417,58808b1eb6a9ebd14974101c600e6f8f,1


### 마지막 구매가 6개월(day 180) 전인 사람

In [17]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
latest_date = df['order_purchase_timestamp'].max()
last_purchase = df.groupby('customer_id')['order_purchase_timestamp'].max().reset_index()
six_months_ago = latest_date - pd.Timedelta(days=180)
churn_customers = last_purchase[last_purchase['order_purchase_timestamp'] <= six_months_ago]
len(churn_customers)
churn_customers

,customer_id,order_purchase_timestamp
2,0000f46a3911fa3c0805444483337064,2023-08-10 16:35:00
3,0000f6ccb0745a6a4b88665a16c9f078,2024-03-12 15:03:00
4,0004aac84e0df4da2b147fca70cf8255,2024-04-14 13:49:00
7,00053a61a98854899e70ed204dd4bafe,2024-07-28 16:30:00
8,0005e1862207bf6ccc02e4228effd9a0,2023-08-04 16:18:00
...,...,...
95412,fffbf87b7a1a6fa8b03f081c5f51a201,2024-05-27 20:04:00
95414,fffcf5a5ff07b0908bd4e2dbc735a684,2023-11-08 15:00:00
95415,fffea47cd6d3cc0a88bd621562a9d061,2024-05-10 12:45:00
95416,ffff371b4d645b6ecea244b27531430a,2023-07-07 04:15:00


### 시간대별로 카테고리1

In [18]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['hour'] = df['order_purchase_timestamp'].dt.hour

In [19]:
def time_period(hour):
    if 6 <= hour < 12:
        return '6~12'
    elif 12 <= hour < 18:
        return '12~18'
    elif 18 <= hour < 24:
        return '18~24'
    else:
        return '0~6'

In [20]:
df['time_period'] = df['hour'].apply(time_period)

In [21]:
result = df.groupby(['time_period', 'category1']).size().reset_index(name='order_count')

In [22]:
result = result.sort_values(['time_period', 'order_count'], ascending=[True, False])

In [23]:
result

,time_period,category1,order_count
2,0~6,식품,2272
3,0~6,의류,1763
1,0~6,스포츠용품,1633
4,0~6,전자제품,1628
5,0~6,화장품,1126
0,0~6,가구,560
8,12~18,식품,9321
9,12~18,의류,7454
10,12~18,전자제품,6704
7,12~18,스포츠용품,6664


In [24]:
result.sort_values

<bound method DataFrame.sort_values of    time_period category1  order_count
2          0~6        식품         2272
3          0~6        의류         1763
1          0~6     스포츠용품         1633
4          0~6      전자제품         1628
5          0~6       화장품         1126
0          0~6        가구          560
8        12~18        식품         9321
9        12~18        의류         7454
10       12~18      전자제품         6704
7        12~18     스포츠용품         6664
11       12~18       화장품         4482
6        12~18        가구         2092
14       18~24        식품        11113
15       18~24        의류         8597
13       18~24     스포츠용품         8026
16       18~24      전자제품         7853
17       18~24       화장품         5346
12       18~24        가구         2531
20        6~12        식품         4029
21        6~12        의류         3209
19        6~12     스포츠용품         2889
22        6~12      전자제품         2810
23        6~12       화장품         1855
18        6~12        가구          942>

### 카테고리1 별 구매량

In [25]:
category1_count = df.groupby('category1')['order_id'].count().reset_index()
category1_count = category1_count.rename(columns={'order_id': 'order_count'})
category1_count = category1_count.sort_values('order_count', ascending=False)
category1_count

,category1,order_count
2,식품,26735
3,의류,21023
1,스포츠용품,19212
4,전자제품,18995
5,화장품,12809
0,가구,6125


### 카테고리2 별 구매량

In [26]:
category2_count = df.groupby('category2')['order_id'].count().reset_index()
category2_count = category2_count.rename(columns={'order_id': 'order_count'})
category2_count = category2_count.sort_values('order_count', ascending=False)
category2_count

,category2,order_count
4,건강식품,9040
22,신선식품,8864
1,가공식품,8831
6,낚세,4852
32,캠핑,4826
11,등산,4824
5,골프,4710
24,여성의류,4294
26,유아의류,4263
21,스포츠의류,4168


### 이탈감지

### 충성고객

### 카테고리별 맞춤고객